In [1]:
# Иvпортируем необходимые библиотеки
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.stats as stats
import random

from scipy.stats import shapiro
from scipy.stats import mannwhitneyu 

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [2]:
df = pd.read_excel(r'..\.\Данные\Данные по объявлениям за 2023-11-01.xlsx')
df.head()

,№,Изменение №,Регион,"Средний срок экспозиции, вторичка",Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 1-ком.,Взвешенная медианная стоимость кв.м. первичка студия Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 1-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 1-ком. Бизнес-класс,...,Количество оферов продажа первичка 1-ком.,Взвешенная медианная стоимость кв.м. вторичка,Взвешенная медианная стоимость кв.м. вторичка 1-ком.,Количество оферов продажа первичка студия Эконом/Комфорт,Количество оферов продажа первичка 1-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком.,Взвешенная медианная стоимость кв.м. вторичка 2-ком.,Взвешенная медианная стоимость кв.м. первичка студия Эконом/Комфорт,Взвешенная медианная стоимость кв.м. вторичка студия
0,1,NaN,Екатеринбург,59 д 21 ч,128 765 ₽,145 650 ₽,254 728 ₽,140 992 ₽,217 293 ₽,270 435 ₽,...,5 558,115 372 ₽,125 806 ₽,2 783,200,136 300 ₽,123 060 ₽,110 337 ₽,152 625 ₽,135 185 ₽
1,2,NaN,Березовский,47 д 2 ч,90 726 ₽,93 911 ₽,—,93 911 ₽,—,—,...,353,102 391 ₽,107 615 ₽,91,—,94 163 ₽,100 743 ₽,102 016 ₽,102 415 ₽,106 607 ₽
2,3,NaN,Верхняя Пышма,52 д,100 793 ₽,104 806 ₽,—,104 806 ₽,—,—,...,386,95 701 ₽,102 940 ₽,58,—,102 957 ₽,95 775 ₽,91 943 ₽,112 000 ₽,112 000 ₽
3,4,NaN,Заречный,18 д 18 ч,—,—,—,—,—,—,...,—,93 110 ₽,98 804 ₽,—,—,—,—,89 866 ₽,—,88 850 ₽
4,5,NaN,Арамиль,150 д 11 ч,—,—,—,—,—,—,...,—,82 193 ₽,89 286 ₽,—,—,—,—,80 769 ₽,—,—


In [3]:
df.shape

(50, 44)

In [4]:
df.columns

Index(['№', 'Изменение №', 'Регион', 'Средний срок экспозиции, вторичка',
       'Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт',
       'Взвешенная медианная стоимость кв.м. первичка 1-ком.',
       'Взвешенная медианная стоимость кв.м. первичка студия Бизнес-класс',
       'Взвешенная медианная стоимость кв.м. первичка 1-ком. Эконом/Комфорт',
       'Взвешенная медианная стоимость кв.м. первичка 3+-ком. Бизнес-класс',
       'Взвешенная медианная стоимость кв.м. первичка 1-ком. Бизнес-класс',
       'Взвешенная медианная стоимость кв.м. первичка 2-ком. Бизнес-класс',
       'Взвешенная медианная стоимость кв.м. первичка 3+-ком. Эконом/Комфорт',
       'Количество оферов продажа первичка 2-ком.',
       'Количество оферов продажа первичка студия.',
       'Количество оферов продажа первичка 3+-ком.',
       'Количество оферов продажа первичка',
       'Количество оферов продажа вторичка студия',
       'Количество оферов продажа вторичка',
       'Количество офер

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 44 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   №                                                                     49 non-null     object 
 1   Изменение №                                                           34 non-null     float64
 2   Регион                                                                48 non-null     object 
 3   Средний срок экспозиции, вторичка                                     48 non-null     object 
 4   Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт   48 non-null     object 
 5   Взвешенная медианная стоимость кв.м. первичка 1-ком.                  48 non-null     object 
 6   Взвешенная медианная стоимость кв.м. первичка студия Бизнес-класс     48 non-null     object 
 7   В

In [6]:
# Выделим в отдельный список даты с месяцами и годами
col_date = ['2023-11',
            '2023-12',
            '2024-01',
            '2024-02',
            '2024-03',
            '2024-04',
            '2024-05',
            '2024-06',
            '2024-07',
            '2024-08',
            '2024-09',
            '2024-10']

In [7]:
# Функция считывания наименования исходных датасетов
def list_name():
    lst_name = list()
    for filename in os.listdir('..\.\Данные'):
        if filename.endswith('.xlsx'):
            lst_name.append("..\.\Данные\\" + filename)
    return lst_name

In [8]:
# Фукнция считывания, первичной обработки и объединения датасетов
def form_dataset(lst: list, column_date: list):
    res = pd.DataFrame()
    for col_dat, name_file in zip(column_date, lst):
        # чтение файла
        df_old = pd.read_excel(name_file) 

        # отсечение первых столбцов и выбор региона Екатеринбург
        df_old = df_old.iloc[:,2:][df['Регион'] == 'Екатеринбург'] 

        # переименование признака Екатеринбург в признак даты
        df_old.loc[df_old['Регион'] == 'Екатеринбург', 'Регион'] = col_dat

        df_old = df_old.rename(columns={'Регион': 'Дата'})

        
        if len(res) == 0:
            res = df_old.copy()
        else:
            res = pd.concat([res, df_old], ignore_index=True)
    return res    

In [9]:
# Считывание списка наименований исходных данных
lst_data = list_name()
lst_data

['..\\.\\Данные\\Данные по объявлениям за 2023-11-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2023-12-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-01-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-02-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-03-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-04-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-05-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-06-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-07-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-08-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-09-01.xlsx',
 '..\\.\\Данные\\Данные по объявлениям за 2024-10-01.xlsx']

In [10]:
# вызов функции и объедиение исходных данных в единый датасет
data = form_dataset(lst_data, col_date)
data

,Дата,"Средний срок экспозиции, вторичка",Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 1-ком.,Взвешенная медианная стоимость кв.м. первичка студия Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 1-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 1-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Эконом/Комфорт,...,Количество оферов продажа первичка 1-ком.,Взвешенная медианная стоимость кв.м. вторичка,Взвешенная медианная стоимость кв.м. вторичка 1-ком.,Количество оферов продажа первичка студия Эконом/Комфорт,Количество оферов продажа первичка 1-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком.,Взвешенная медианная стоимость кв.м. вторичка 2-ком.,Взвешенная медианная стоимость кв.м. первичка студия Эконом/Комфорт,Взвешенная медианная стоимость кв.м. вторичка студия
0,2023-11,59 д 21 ч,128 765 ₽,145 650 ₽,254 728 ₽,140 992 ₽,217 293 ₽,270 435 ₽,242 553 ₽,118 894 ₽,...,5 558,115 372 ₽,125 806 ₽,2 783,200,136 300 ₽,123 060 ₽,110 337 ₽,152 625 ₽,135 185 ₽
1,2023-12,66 д 19 ч,132 020 ₽,146 796 ₽,256 448 ₽,143 992 ₽,216 770 ₽,259 135 ₽,238 224 ₽,122 449 ₽,...,6 775,117 591 ₽,127 519 ₽,3 026,165,138 974 ₽,124 790 ₽,112 150 ₽,155 365 ₽,140 011 ₽
2,2024-01,79 д 6 ч,135 520 ₽,150 670 ₽,244 447 ₽,148 685 ₽,220 803 ₽,276 576 ₽,242 159 ₽,125 874 ₽,...,7 086,118 802 ₽,129 037 ₽,3 380,110,142 641 ₽,130 605 ₽,112 407 ₽,155 000 ₽,140 000 ₽
3,2024-02,57 д,138 485 ₽,153 015 ₽,244 447 ₽,150 548 ₽,216 001 ₽,287 999 ₽,240 094 ₽,124 695 ₽,...,7 242,119 015 ₽,128 479 ₽,3 216,130,145 095 ₽,128 823 ₽,113 475 ₽,162 103 ₽,139 442 ₽
4,2024-03,59 д 19 ч,140 058 ₽,154 539 ₽,247 746 ₽,151 718 ₽,221 604 ₽,304 492 ₽,262 304 ₽,126 616 ₽,...,6 768,119 704 ₽,129 590 ₽,2 661,125,146 331 ₽,131 141 ₽,113 317 ₽,162 999 ₽,142 269 ₽
5,2024-04,56 д 13 ч,142 043 ₽,153 098 ₽,255 592 ₽,150 630 ₽,240 021 ₽,285 360 ₽,247 604 ₽,129 085 ₽,...,6 768,120 571 ₽,129 858 ₽,2 887,124,146 205 ₽,133 961 ₽,113 636 ₽,159 709 ₽,144 840 ₽
6,2024-05,75 д 16 ч,142 587 ₽,155 010 ₽,268 660 ₽,153 000 ₽,218 243 ₽,273 734 ₽,259 604 ₽,128 925 ₽,...,6 846,119 025 ₽,129 052 ₽,2 538,114,148 227 ₽,133 556 ₽,112 773 ₽,164 437 ₽,142 334 ₽
7,2024-06,56 д 21 ч,144 713 ₽,159 786 ₽,259 519 ₽,158 412 ₽,232 608 ₽,274 423 ₽,252 024 ₽,133 000 ₽,...,6 925,120 334 ₽,130 756 ₽,2 499,82,152 637 ₽,136 037 ₽,113 065 ₽,171 558 ₽,143 177 ₽
8,2024-07,62 д 19 ч,144 089 ₽,158 925 ₽,226 542 ₽,157 146 ₽,221 136 ₽,281 633 ₽,235 000 ₽,135 253 ₽,...,6 298,121 593 ₽,131 429 ₽,2 913,90,152 105 ₽,137 300 ₽,114 022 ₽,167 515 ₽,144 444 ₽
9,2024-08,72 д 9 ч,144 767 ₽,159 721 ₽,254 486 ₽,156 278 ₽,216 998 ₽,271 511 ₽,235 000 ₽,135 847 ₽,...,5 924,122 278 ₽,134 146 ₽,2 754,177,151 924 ₽,142 343 ₽,115 012 ₽,167 213 ₽,144 902 ₽


In [11]:
# Удалим признаки
data.drop(['Дата',
          'Взвешенная медианная стоимость кв.м. первичка 1-ком.',
          'Взвешенная медианная стоимость кв.м. первичка 1-ком. Эконом/Комфорт',
          'Взвешенная медианная стоимость кв.м. первичка 1-ком. Бизнес-класс',
          'Взвешенная медианная стоимость кв.м. первичка студия Бизнес-класс',
          'Количество оферов продажа первичка студия.',
          'Количество оферов продажа первичка',
          'Количество оферов продажа вторичка студия',
          'Количество оферов продажа вторичка',
          'Количество оферов продажа вторичка 1-ком.',
          'Количество оферов продажа первичка Бизнес-класс',
          'Количество оферов продажа первичка Эконом/Комфорт',
          'Количество оферов продажа первичка студия Бизнес-класс',
          'Количество оферов продажа первичка 1-ком. Эконом/Комфорт',
          'Взвешенная медианная стоимость кв.м. первичка Бизнес-класс',
          'Взвешенная медианная стоимость кв.м. первичка студия.',
          'Взвешенная медианная стоимость кв.м. первичка',
          'Количество оферов продажа первичка 1-ком.',
          'Взвешенная медианная стоимость кв.м. вторичка',
          'Взвешенная медианная стоимость кв.м. вторичка 1-ком.',
          'Количество оферов продажа первичка студия Эконом/Комфорт',
          'Количество оферов продажа первичка 1-ком. Бизнес-класс',
          'Взвешенная медианная стоимость кв.м. первичка Эконом/Комфорт',
          'Взвешенная медианная стоимость кв.м. первичка студия Эконом/Комфорт',
          'Взвешенная медианная стоимость кв.м. вторичка студия'],
          axis=1, inplace=True)
data

,"Средний срок экспозиции, вторичка",Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Эконом/Комфорт,Количество оферов продажа первичка 2-ком.,Количество оферов продажа первичка 3+-ком.,Количество оферов продажа первичка 3+-ком. Бизнес-класс,Количество оферов продажа вторичка 3+-ком.,Взвешенная медианная стоимость кв.м. первичка 2-ком.,Количество оферов продажа первичка 3+-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. вторичка 3+-ком.,Количество оферов продажа вторичка 2-ком.,Количество оферов продажа первичка 2-ком. Эконом/Комфорт,Количество оферов продажа первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком.,Взвешенная медианная стоимость кв.м. вторичка 2-ком.
0,59 д 21 ч,128 765 ₽,217 293 ₽,242 553 ₽,118 894 ₽,4 924,2 055,87,2 003,131 169 ₽,1 968,104 670 ₽,2 542,4 820,104,123 060 ₽,110 337 ₽
1,66 д 19 ч,132 020 ₽,216 770 ₽,238 224 ₽,122 449 ₽,6 319,2 458,61,1 803,133 919 ₽,2 397,105 129 ₽,2 346,6 206,113,124 790 ₽,112 150 ₽
2,79 д 6 ч,135 520 ₽,220 803 ₽,242 159 ₽,125 874 ₽,6 427,2 448,122,1 864,137 478 ₽,2 326,106 767 ₽,2 514,6 309,118,130 605 ₽,112 407 ₽
3,57 д,138 485 ₽,216 001 ₽,240 094 ₽,124 695 ₽,6 644,2 588,117,1 773,140 458 ₽,2 471,106 893 ₽,2 491,6 515,129,128 823 ₽,113 475 ₽
4,59 д 19 ч,140 058 ₽,221 604 ₽,262 304 ₽,126 616 ₽,6 168,2 246,107,1 865,142 357 ₽,2 139,107 407 ₽,2 601,6 052,116,131 141 ₽,113 317 ₽
5,56 д 13 ч,142 043 ₽,240 021 ₽,247 604 ₽,129 085 ₽,6 874,2 457,108,1 881,143 640 ₽,2 349,107 971 ₽,2 601,6 770,104,133 961 ₽,113 636 ₽
6,75 д 16 ч,142 587 ₽,218 243 ₽,259 604 ₽,128 925 ₽,6 144,2 141,111,1 888,144 473 ₽,2 030,105 941 ₽,2 626,6 045,99,133 556 ₽,112 773 ₽
7,56 д 21 ч,144 713 ₽,232 608 ₽,252 024 ₽,133 000 ₽,6 215,2 001,61,1 857,145 904 ₽,1 940,108 062 ₽,2 541,6 146,69,136 037 ₽,113 065 ₽
8,62 д 19 ч,144 089 ₽,221 136 ₽,235 000 ₽,135 253 ₽,5 720,1 888,45,1 827,145 536 ₽,1 843,111 388 ₽,2 520,5 629,91,137 300 ₽,114 022 ₽
9,72 д 9 ч,144 767 ₽,216 998 ₽,235 000 ₽,135 847 ₽,5 668,1 899,152,1 739,148 158 ₽,1 747,110 953 ₽,2 433,5 455,213,142 343 ₽,115 012 ₽


In [12]:
# переведем в единый формат признак Средний срок экспозиции 
data['Средний срок экспозиции, вторичка'] = data['Средний срок экспозиции, вторичка'].str.split().apply(lambda x: int(x[0]) * 24 + int(x[2]) if len(x) == 4 else int(x[0]) * 24)
data

,"Средний срок экспозиции, вторичка",Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Эконом/Комфорт,Количество оферов продажа первичка 2-ком.,Количество оферов продажа первичка 3+-ком.,Количество оферов продажа первичка 3+-ком. Бизнес-класс,Количество оферов продажа вторичка 3+-ком.,Взвешенная медианная стоимость кв.м. первичка 2-ком.,Количество оферов продажа первичка 3+-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. вторичка 3+-ком.,Количество оферов продажа вторичка 2-ком.,Количество оферов продажа первичка 2-ком. Эконом/Комфорт,Количество оферов продажа первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком.,Взвешенная медианная стоимость кв.м. вторичка 2-ком.
0,1437,128 765 ₽,217 293 ₽,242 553 ₽,118 894 ₽,4 924,2 055,87,2 003,131 169 ₽,1 968,104 670 ₽,2 542,4 820,104,123 060 ₽,110 337 ₽
1,1603,132 020 ₽,216 770 ₽,238 224 ₽,122 449 ₽,6 319,2 458,61,1 803,133 919 ₽,2 397,105 129 ₽,2 346,6 206,113,124 790 ₽,112 150 ₽
2,1902,135 520 ₽,220 803 ₽,242 159 ₽,125 874 ₽,6 427,2 448,122,1 864,137 478 ₽,2 326,106 767 ₽,2 514,6 309,118,130 605 ₽,112 407 ₽
3,1368,138 485 ₽,216 001 ₽,240 094 ₽,124 695 ₽,6 644,2 588,117,1 773,140 458 ₽,2 471,106 893 ₽,2 491,6 515,129,128 823 ₽,113 475 ₽
4,1435,140 058 ₽,221 604 ₽,262 304 ₽,126 616 ₽,6 168,2 246,107,1 865,142 357 ₽,2 139,107 407 ₽,2 601,6 052,116,131 141 ₽,113 317 ₽
5,1357,142 043 ₽,240 021 ₽,247 604 ₽,129 085 ₽,6 874,2 457,108,1 881,143 640 ₽,2 349,107 971 ₽,2 601,6 770,104,133 961 ₽,113 636 ₽
6,1816,142 587 ₽,218 243 ₽,259 604 ₽,128 925 ₽,6 144,2 141,111,1 888,144 473 ₽,2 030,105 941 ₽,2 626,6 045,99,133 556 ₽,112 773 ₽
7,1365,144 713 ₽,232 608 ₽,252 024 ₽,133 000 ₽,6 215,2 001,61,1 857,145 904 ₽,1 940,108 062 ₽,2 541,6 146,69,136 037 ₽,113 065 ₽
8,1507,144 089 ₽,221 136 ₽,235 000 ₽,135 253 ₽,5 720,1 888,45,1 827,145 536 ₽,1 843,111 388 ₽,2 520,5 629,91,137 300 ₽,114 022 ₽
9,1737,144 767 ₽,216 998 ₽,235 000 ₽,135 847 ₽,5 668,1 899,152,1 739,148 158 ₽,1 747,110 953 ₽,2 433,5 455,213,142 343 ₽,115 012 ₽


In [13]:
# Обработка признаков
def convert_to_int(data):
    for col in data.select_dtypes('object'):
        data[col] = data[col].str.replace('₽', '', regex=True).replace(' ','',regex=True).astype('int')
    return data    

In [14]:
# применим функцию конвертации данных
data = convert_to_int(data)
data

,"Средний срок экспозиции, вторичка",Взвешенная медианная стоимость кв.м. первичка 2-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком. Эконом/Комфорт,Количество оферов продажа первичка 2-ком.,Количество оферов продажа первичка 3+-ком.,Количество оферов продажа первичка 3+-ком. Бизнес-класс,Количество оферов продажа вторичка 3+-ком.,Взвешенная медианная стоимость кв.м. первичка 2-ком.,Количество оферов продажа первичка 3+-ком. Эконом/Комфорт,Взвешенная медианная стоимость кв.м. вторичка 3+-ком.,Количество оферов продажа вторичка 2-ком.,Количество оферов продажа первичка 2-ком. Эконом/Комфорт,Количество оферов продажа первичка 2-ком. Бизнес-класс,Взвешенная медианная стоимость кв.м. первичка 3+-ком.,Взвешенная медианная стоимость кв.м. вторичка 2-ком.
0,1437,128765,217293,242553,118894,4924,2055,87,2003,131169,1968,104670,2542,4820,104,123060,110337
1,1603,132020,216770,238224,122449,6319,2458,61,1803,133919,2397,105129,2346,6206,113,124790,112150
2,1902,135520,220803,242159,125874,6427,2448,122,1864,137478,2326,106767,2514,6309,118,130605,112407
3,1368,138485,216001,240094,124695,6644,2588,117,1773,140458,2471,106893,2491,6515,129,128823,113475
4,1435,140058,221604,262304,126616,6168,2246,107,1865,142357,2139,107407,2601,6052,116,131141,113317
5,1357,142043,240021,247604,129085,6874,2457,108,1881,143640,2349,107971,2601,6770,104,133961,113636
6,1816,142587,218243,259604,128925,6144,2141,111,1888,144473,2030,105941,2626,6045,99,133556,112773
7,1365,144713,232608,252024,133000,6215,2001,61,1857,145904,1940,108062,2541,6146,69,136037,113065
8,1507,144089,221136,235000,135253,5720,1888,45,1827,145536,1843,111388,2520,5629,91,137300,114022
9,1737,144767,216998,235000,135847,5668,1899,152,1739,148158,1747,110953,2433,5455,213,142343,115012


In [15]:
# сгенерируем новые данные, т.к. данных очень мало
for _ in range(1000):
    lst = list()
    for col in data.columns:
        lst.append(int(random.randrange(data[col].min(), data[col].max()) + data[col].std()))
    data.loc[data.shape[0]] = lst

In [16]:
# разделим данные на обучающую и тестовую выборку

scaler = StandardScaler()

X = data.drop('Средний срок экспозиции, вторичка', axis=1)
y = data['Средний срок экспозиции, вторичка']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

X_train = pd.DataFrame(X_train_scaler, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaler, columns=X_test.columns)

#### Обучение моделей регрессии

In [17]:
# обучим модель случайного леса с параметрами по умолчанию
model_rf = RandomForestRegressor(random_state=9)

model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_rf))
print('MSE: ', mean_squared_error(y_test, y_pred_rf))
print('MAE: ', mean_absolute_error(y_test, y_pred_rf))

R2:  0.8902639306291835
MSE:  3.0197788338547304e+16
MAE:  69327685.06995074


In [18]:
# обучим модель деревья решений
model_dt = DecisionTreeRegressor(random_state=9)

model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_dt))
print('MSE: ', mean_squared_error(y_test, y_pred_dt))
print('MAE: ', mean_absolute_error(y_test, y_pred_dt))

R2:  0.6076014242077183
MSE:  1.0798244555380475e+17
MAE:  115354491.33990148


In [19]:
# обучим гребневую модель 
model_ridge = RidgeCV()

model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_ridge))
print('MSE: ', mean_squared_error(y_test, y_pred_ridge))
print('MAE: ', mean_absolute_error(y_test, y_pred_ridge))

R2:  0.8167277434484042
MSE:  5.043388963542437e+16
MAE:  80218552.1304744


In [20]:
# обучим модель Лассо
model_lasso = LassoCV(random_state=9)

model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_lasso))
print('MSE: ', mean_squared_error(y_test, y_pred_lasso))
print('MAE: ', mean_absolute_error(y_test, y_pred_lasso))

R2:  0.8613639695746359
MSE:  3.815064205311257e+16
MAE:  75624646.28823365


#### Подбор гиперпараметров

In [25]:
# для улучшения качество моделей, воспользуемся методом подбора гиперпараметров для случайного дерева

param_grid = {'n_estimators': np.arange(100, 300, 100),
              'max_depth': np.arange(3, 20, 2),
             'min_samples_split': np.arange(2, 20, 2),
             'min_samples_leaf': np.arange(2, 20, 2)}

rf_search = GridSearchCV(model_rf, param_grid=param_grid, scoring='r2', n_jobs=-1)
rf_search.fit(X_train, y_train)

print(f'Наилучшие гиперпараметры для модели случайного дерева: {rf_search.best_params_}')
print(f'Лучшая метрика r2: {rf_search.best_score_}')

Наилучшие гиперпараметры для модели случайного дерева: {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Лучшая метрика r2: 0.7819478922491133


In [26]:
# для улучшения качество моделей, воспользуемся методом подбора гиперпараметров для деревья решений

param_grid = {'max_features': np.arange(100, 300, 100),
              'max_depth': np.arange(3, 20, 2),
             'min_samples_split': np.arange(2, 20, 2),
             'min_samples_leaf': np.arange(2, 20, 2)}

dt_search = GridSearchCV(model_dt, param_grid=param_grid, scoring='r2', n_jobs=-1)
dt_search.fit(X_train, y_train)

print(f'Наилучшие гиперпараметры для модели случайного дерева: {dt_search.best_params_}')
print(f'Лучшая метрика r2: {dt_search.best_score_}')

Наилучшие гиперпараметры для модели случайного дерева: {'max_depth': 13, 'max_features': 100, 'min_samples_leaf': 16, 'min_samples_split': 2}
Лучшая метрика r2: 0.7193699423986445


In [32]:
# для улучшения качество моделей, воспользуемся методом подбора гиперпараметров для гребневой регрессии

param_grid = {'alphas': np.random.uniform(0.1, 2, 100),
             "fit_intercept": [True, False],
             'gcv_mode': ['auto', 'svd', 'eigen']}

ridge_search = GridSearchCV(model_ridge, param_grid=param_grid, scoring='r2', n_jobs=-1)
ridge_search.fit(X_train, y_train)

print(f'Наилучшие гиперпараметры для модели гребневой регрессии: {ridge_search.best_params_}')
print(f'Лучшая метрика r2: {ridge_search.best_score_}')

Наилучшие гиперпараметры для модели гребневой регрессии: {'alphas': 1.9777020357538824, 'fit_intercept': True, 'gcv_mode': 'auto'}
Лучшая метрика r2: 0.7023993050611803


In [47]:
# для улучшения качество моделей, воспользуемся методом подбора гиперпараметров для Лассо регрессии

param_grid = {'max_iter': np.arange(1000, 10_000, 100),
             'selection': ['cyclic', 'random'],
             'precompute': [True, False, 'auto'],
             'n_alphas': np.arange(100, 1000, 100)}

lasso_search = GridSearchCV(model_lasso, param_grid=param_grid, scoring='r2', n_jobs=-1)
lasso_search.fit(X_train, y_train)

print(f'Наилучшие гиперпараметры для модели Лассо: {lasso_search.best_params_}')
print(f'Лучшая метрика r2: {lasso_search.best_score_}')

Наилучшие гиперпараметры для модели Лассо: {'max_iter': 4100, 'n_alphas': 200, 'precompute': True, 'selection': 'random'}
Лучшая метрика r2: 0.726759908976116


#### Оценка моделей на подобранных гиперпараметрах

In [21]:
# обучим модель случайного леса с гиперпараметрами
model_rf = RandomForestRegressor(max_depth= 3, 
                                 min_samples_leaf= 2, 
                                 min_samples_split= 2, 
                                 n_estimators= 100,
                                 random_state=9)

model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_rf))
print('MSE: ', mean_squared_error(y_test, y_pred_rf))
print('MAE: ', mean_absolute_error(y_test, y_pred_rf))

R2:  0.8876096072396763
MSE:  3.0928219967436224e+16
MAE:  77426543.57310024


In [22]:
# обучим модель деревья решений
model_dt = DecisionTreeRegressor(max_depth= 13, 
                                 max_features= 100,
                                 min_samples_leaf= 16, 
                                 min_samples_split= 2,
                                 random_state=9)

model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_dt))
print('MSE: ', mean_squared_error(y_test, y_pred_dt))
print('MAE: ', mean_absolute_error(y_test, y_pred_dt))

R2:  0.8289504176953416
MSE:  4.7070385438889064e+16
MAE:  81058062.24451694


In [23]:
# обучим гребневую модель 
model_ridge = RidgeCV(alphas= 1.9777020357538824,
                      fit_intercept= True,
                      gcv_mode= 'auto')

model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_ridge))
print('MSE: ', mean_squared_error(y_test, y_pred_ridge))
print('MAE: ', mean_absolute_error(y_test, y_pred_ridge))

R2:  0.8079455573710761
MSE:  5.285062096026091e+16
MAE:  81966301.57354033


In [24]:
# обучим модель Лассо
model_lasso = LassoCV(max_iter= 4100,
                      n_alphas= 200,
                      precompute= True,
                      selection='random')

model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test)

print('R2: ', r2_score(y_test, y_pred_lasso))
print('MSE: ', mean_squared_error(y_test, y_pred_lasso))
print('MAE: ', mean_absolute_error(y_test, y_pred_lasso))

R2:  0.8614934063379286
MSE:  3.811502291709329e+16
MAE:  75654421.38375522


#### Попробуем применить ансамблевый метод 

In [25]:
# Стекинг с подобранными гиперпараметрами

estimators = [('rf', RandomForestRegressor(max_depth= 3,min_samples_leaf= 2,min_samples_split= 2,n_estimators= 100)),
             ('dt', DecisionTreeRegressor(max_depth= 13,max_features= 100,min_samples_leaf= 16,min_samples_split= 2)),
             ('rd', RidgeCV(alphas= 1.9777020357538824,fit_intercept= True,gcv_mode= 'auto')),
             ('ls', LassoCV(max_iter= 4100,n_alphas= 200,precompute= True,selection='random'))]



modelClf = StackingRegressor(estimators=estimators,final_estimator=KNeighborsRegressor(n_neighbors=100), cv=3, n_jobs=-1)

modelClf.fit(X_train, y_train)
y_pred_Clf = modelClf.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Clf))
print('MSE: ', mean_squared_error(y_test, y_pred_Clf))
print('MAE: ', mean_absolute_error(y_test, y_pred_Clf))

R2:  0.8013985790132206
MSE:  5.465225526191877e+16
MAE:  85328377.53527091


In [26]:
# Стекинг с параметрами по умолчанию

estimators = [('rf', RandomForestRegressor()),
             ('dt', DecisionTreeRegressor()),
             ('rd', RidgeCV()),
             ('ls', LassoCV())]

modelClf = StackingRegressor(estimators=estimators,final_estimator=KNeighborsRegressor(), cv=3, n_jobs=-1)

modelClf.fit(X_train, y_train)
y_pred_Clf = modelClf.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Clf))
print('MSE: ', mean_squared_error(y_test, y_pred_Clf))
print('MAE: ', mean_absolute_error(y_test, y_pred_Clf))

R2:  0.8138875037702284
MSE:  5.121548275356738e+16
MAE:  81747582.74482758


In [27]:
# Адаптивный бустинг моделей с параметрами по умолчанию

modelDT = AdaBoostRegressor(estimator=DecisionTreeRegressor(), n_estimators=100)
modelDT.fit(X_train, y_train)

y_pred_DT = modelDT.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_DT))
print('MSE: ', mean_squared_error(y_test, y_pred_DT))
print('MAE: ', mean_absolute_error(y_test, y_pred_DT))

R2:  0.85779418595957
MSE:  3.9132995172194136e+16
MAE:  74872895.97536945


In [28]:
modelRf = AdaBoostRegressor(estimator=RandomForestRegressor(), n_estimators=100)
modelRf.fit(X_train, y_train)

y_pred_Rf = modelRf.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Rf))
print('MSE: ', mean_squared_error(y_test, y_pred_Rf))
print('MAE: ', mean_absolute_error(y_test, y_pred_Rf))

R2:  0.8732581135636471
MSE:  3.4877544659450256e+16
MAE:  85481426.17374386


In [29]:
modelRd = AdaBoostRegressor(estimator=RidgeCV(), n_estimators=100)
modelRd.fit(X_train, y_train)

y_pred_Rd = modelRd.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Rd))
print('MSE: ', mean_squared_error(y_test, y_pred_Rd))
print('MAE: ', mean_absolute_error(y_test, y_pred_Rd))

R2:  0.7078119726932124
MSE:  8.04059435904543e+16
MAE:  100901908.06215915


In [30]:
modelLs = AdaBoostRegressor(estimator=LassoCV(), n_estimators=100)
modelLs.fit(X_train, y_train)

y_pred_Ls = modelLs.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Ls))
print('MSE: ', mean_squared_error(y_test, y_pred_Ls))
print('MAE: ', mean_absolute_error(y_test, y_pred_Ls))

R2:  0.7288193478785071
MSE:  7.462501601548973e+16
MAE:  99795243.64466166


In [31]:
# Адаптивный бустинг моделей с подобранными гиперпараметрами

modelDT = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth= 13,max_features= 100,min_samples_leaf= 16,min_samples_split= 2), n_estimators=100)
modelDT.fit(X_train, y_train)

y_pred_DT = modelDT.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_DT))
print('MSE: ', mean_squared_error(y_test, y_pred_DT))
print('MAE: ', mean_absolute_error(y_test, y_pred_DT))

R2:  0.7911494078234299
MSE:  5.7472679795159336e+16
MAE:  192281302.35702318


In [32]:
modelRf = AdaBoostRegressor(estimator=RandomForestRegressor(max_depth= 3,min_samples_leaf= 2,min_samples_split= 2,n_estimators= 100), n_estimators=100)
modelRf.fit(X_train, y_train)

y_pred_Rf = modelRf.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Rf))
print('MSE: ', mean_squared_error(y_test, y_pred_Rf))
print('MAE: ', mean_absolute_error(y_test, y_pred_Rf))

R2:  0.8614428625620252
MSE:  3.812893183742654e+16
MAE:  84876742.0832052


In [33]:
modelRd = AdaBoostRegressor(estimator=RidgeCV(alphas= 1.9777020357538824,fit_intercept= True,gcv_mode= 'auto'), n_estimators=100)
modelRd.fit(X_train, y_train)

y_pred_Rd = modelRd.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Rd))
print('MSE: ', mean_squared_error(y_test, y_pred_Rd))
print('MAE: ', mean_absolute_error(y_test, y_pred_Rd))

R2:  0.7568963373260685
MSE:  6.6898632253228136e+16
MAE:  97797481.35869485


In [35]:
modelLs = AdaBoostRegressor(estimator=LassoCV(max_iter= 4100,n_alphas= 200,precompute= True,selection='random'), n_estimators=100)
modelLs.fit(X_train, y_train)

y_pred_Ls = modelLs.predict(X_test)

print('R2: ',r2_score(y_test, y_pred_Ls))
print('MSE: ', mean_squared_error(y_test, y_pred_Ls))
print('MAE: ', mean_absolute_error(y_test, y_pred_Ls))

R2:  0.7964689173014846
MSE:  5.600882727880601e+16
MAE:  87195839.8635048
